In [261]:
import numpy as np
import math
import gurobipy as gp
from gurobipy import GRB

### Preprocessing

TODO:
* Define $u_i$, utility functions
* Define $c$, course capacities
* Define $b_0$, initial budgets    
* Define $K$, max. bundle size

In [262]:
# K is max. number of classes
K = 2
c = [1, 1, 1, 1]
b = [1, 1]
u = [[10, 5, 4, 3]]

In [263]:
# combinations of length n
def combinations(array, tuple_length, prev_array=[]):
    if len(prev_array) == tuple_length:
        return [prev_array]
    combs = []
    for i, val in enumerate(array):
        prev_array_extended = prev_array.copy()
        prev_array_extended.append(val)
        combs += combinations(array[i+1:], tuple_length, prev_array_extended)
    return combs

# all bundles below a maximum length. Greedy version, probably a better way exists
def powerset(omega, max_length):
    subsets = []
    for i in range(max_length):
        subsets += combinations(omega, i+1)
    return subsets

# turns a bundle into corresponding allocation vector in R^m
def bundle_to_allocation(bundle):
    allocation = np.zeros(M)
    for i in bundle:
        allocation[i-1] = 1
    return allocation

# costs for an allocation vector
def cost(allocation, prices):
    return np.dot(allocation, prices)

In [264]:
# Input: Max bundle size
# Output: set of possible allocation vectors
def allocations(max_length):
    omega = np.array(range(1, M+1))
    feasible_bundles = powerset(omega, max_length)
    allocations = []
    for i in feasible_bundles:
        allocations.append(bundle_to_allocation(i))
    return allocations

In [265]:
# input: set of allocations, utility, prices, and budget
# output: utility-maximizing budget-feasible bundles

# note: not always unique
def demand_set(allocations, prices, utility, budget):
    demand_set = []

    max_u = 0
    for i in allocations:
        feasible = (cost(i, prices) <= budget)
        if feasible:
            u = np.dot(i, utility)
            if u < max_u:
                continue
            elif u == max_u:
                demand_set.append(i)
            elif u > max_u:
                max_u = u
                demand_set.clear()
                demand_set.append(i)
        
    return np.array(demand_set)

In [266]:
#Please make e/d a whole number!

def budget_intervals(bundles, prices, utility, budget, delta, epsilon):
    discontinuities = []
    allocations = []
    lower = budget-epsilon
    discontinuities.append(lower)
    current_demand = demand_set(bundles, prices, utility, lower)

    #find closest multiple of delta to lower
    current_budget = lower

    while (current_budget < budget+epsilon):
        new_budget = current_budget + delta
        new_demand = demand_set(bundles, prices, utility, new_budget)
        if (np.array_equal(new_demand, current_demand)):
            pass
        else:
            discontinuities.append(new_budget)
            allocations.append(current_demand)
            current_demand = new_demand
        current_budget += delta

    discontinuities.append(budget+epsilon)
    allocations.append(demand_set(bundles, prices, utility, budget+epsilon))

    
    #Get intervals from discontinuity points
    intervals = []
    for i in range(len(discontinuities) - 1):
        intervals.append([discontinuities[i], discontinuities[i+1]])
    
    #return intervals, allocations
    return intervals, allocations

## A-CEEI

Inputs 
* Student's utility functions $u_i$
* Course capacities $c$
* Initial budgets b_0

Parameters
* Step size $\delta$
* Max. budget perturbation $\epsilon$

Outputs
* Final prices $p^* \in \mathbb{R}^m$
* Final budgets $b^* \in \mathbb{R}^n$

### Description

1) Take in $u, c, b_0$.
2) Set p = 0.
3) Find optimal budget perturbation given current prices.
4) With this set of budgets, compute clearing error.
5) Terminate if clearing error is 0. Otherwise, update prices.

In [267]:
# step size and error 
# a = allocations
# c = course capacities

# p = prices

def clearing_error_optimizer(a, c, p):
	n=len(a)
	m=len(c)
	ki=len(a[0])
	# Create a new model
	model = gp.Model("Clearing_Error_Minimization")

	# Decision variables
	# These are the dimensions of the decision variable array
	# array of binary decision variables with n rows and ki columns
	z = model.addVars(m, lb=0.0, vtype=GRB.INTEGER, name="z")
	x = model.addVars(n, ki, vtype=GRB.BINARY, name="x")
	print(z)

	# Objective function: Minimize the l1-norm of vector z
	model.setObjective(gp.quicksum(z[j] for j in range(m)), sense=GRB.MINIMIZE)

	# Constraints
	for j in range(m):
		if p[j]>0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) == c[j] + z[j], f"clearing_error_positive_{j}")
		if p[j]==0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) <= c[j] + z[j], f"clearing_error_nonnegative_{j}")
	
	for i in range(n):
		model.addConstr(gp.quicksum(x[i, l] for l in range(ki)) == 1, f"one_schedule_per_student_{i}")

	# Solve the model
	model.optimize()

	# Check optimization status
	if model.status == GRB.OPTIMAL:
		# Access the optimal solution
		optimal_x = model.getAttr("x", x)
		optimal_z = model.getAttr("x", z)

		# Print or use the optimal solution as needed
		print("Optimal x values:", optimal_x)
		print("Optimal z values:", optimal_z)
		return (optimal_x, optimal_z)
	else:
		print(f"Optimization terminated with status {model.status}")
		# Check if the model is infeasible
		print(f"HELLO")
		model.computeIIS()
		print("\nInfeasible constraints:")
		for c in model.getConstrs():
			if c.IISConstr:
				print(c.constrName)
		return (None, None)

In [268]:
def ACEEI(u, c, b_0, K, delta, epsilon):
    p = np.zeros(len(c))
    bundles = allocations(max_length=K)
    
    zeroClearingError = False

    while(not zeroClearingError):
        a = []
        for i in range(len(u)):
            demands = budget_intervals(bundles, p, u[i], b_0[i], delta, epsilon)
            a.append(demands)

        x, z = clearing_error_optimizer(a, c, p)

        error = list(z.values())

        zeroClearingError = True
        for i in error:
            if (i != 0):
                zeroClearingError = False
        
        if (zeroClearingError):
            print("\n\n\n\nExact equilibrium found")
            return p, x, z
        else:
            p = np.add(p, delta*np.array(list(z.values())))

    return p, x, z

In [269]:
small = ACEEI(u, c, b, 2, .5, 1)
print(small)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>}


TypeError: can't multiply sequence by non-int of type 'Var'